In [2]:
%cd ..
import ast
import numpy as np
import pandas as pd
import warnings
from winedge import Card, Evaluator

\\wsl.localhost\Ubuntu\home\jtatka\WinningEdge


In [3]:
#Reading data from original dataframe.pkl file
df = pd.read_pickle('dataframe/poker_dataframe_bugfixed.pkl')
#Clean dataframe to just games which completed and have river actions
blf_df = df[(~df['River'].isnull()) & (~df['SB cards'].isnull()) & \
            (~df['BB cards'].isnull()) & (~df['River actions'].isnull())]

#Add Columns to dataframe for SB & BB agg and bluff
warnings.filterwarnings('ignore')
blf_df['aggSB'] = np.nan
blf_df['aggBB'] = np.nan
blf_df['SB Bluff Metric'] = np.nan
blf_df['BB Bluff Metric'] = np.nan
blf_df = blf_df.reset_index()
blf_df

,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,...,River,SB stack,BB stack,SB cards,BB cards,Folded pre,aggSB,aggBB,SB Bluff Metric,BB Bluff Metric
0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]","[5c, 9c, 5s]",...,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]",False,NaN,NaN,NaN,NaN
1,9,603607691,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,MARUSYA94,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[9d, 4d, 9s]",...,6h,1.20,0.80,"[ 5c, 4c ]","[ 3h, 7s ]",False,NaN,NaN,NaN,NaN
2,14,603607964,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,gunchozaurs,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[Qs, 8s, 5s]",...,4c,2.63,2.01,"[ Kh, 9c ]","[ 2c, Jh ]",False,NaN,NaN,NaN,NaN
3,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[6s, 7s, 5h]",...,2d,2.65,2,"[ 5c, Ad ]","[ 6h, 9h ]",False,NaN,NaN,NaN,NaN
4,33,1180858754,Berkeley-0.01-0.02-USD-NoLimitHoldem-Pacific-6...,trebs15,turron3618,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[5d, Qd, 6d]",...,Jd,2.47,0.72,"[ 4c, Qh ]","[ 9h, 8c ]",False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77646,520797,553224321,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]","[6c, Jc, 3d]",...,9c,0.96,0.89,"[ 5d, Jh ]","[ Ad, As ]",False,NaN,NaN,NaN,NaN
77647,520798,553224342,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (r, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[7s, Jd, 5h]",...,Js,1,1,"[ 9s, Qd ]","[ 6c, Ac ]",False,NaN,NaN,NaN,NaN
77648,520799,553224357,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,roghnov,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[Jd, As, Qd]",...,Ts,1.04,0.96,"[ 9s, Ad ]","[ Qh, 7s ]",False,NaN,NaN,NaN,NaN
77649,520800,553224367,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[4d, 2d, Js]",...,Ts,1.10,0.90,"[ Qc, Kc ]","[ 9s, Ks ]",False,NaN,NaN,NaN,NaN


In [ ]:
#FINAL Calc Aggressiveness
# Aggressiveness baseline is 1. Player aggressiveness is increased by
# one every time they bet 'b' or raise 'r' throughout the river action phase.


for loc in blf_df.index:
    sb_agg = 1
    bb_agg = 1
    for i, action in enumerate(blf_df['River actions'][loc]):
        if i %2==0:
            if action[0] == 'b' or action[0] == 'r':
                sb_agg += 1
            else:
                sb_agg += 0
        else:
            if action[0] == 'b' or action[0] == 'r':
                bb_agg += 1
            else:
                bb_agg += 0
    blf_df['aggBB'][loc] = bb_agg
    blf_df['aggSB'][loc] = sb_agg

#Normalize aggressiveness to mean aggressiveness for player in that position 
# ie small blind, big blind
blf_df['aggSB'] = blf_df['aggSB']/blf_df['aggSB'].mean()
blf_df['aggBB'] = blf_df['aggBB']/blf_df['aggBB'].mean()
blf_df

In [85]:
# Calc bluff metric ([1-%rank] * normalized aggressiveness)
# Using 1-% rank to quantify the percentage of hands that their hand loses too.
# will mean that a higher bluff metric equals more bets with low likelihood of winning

for loc in blf_df.index:
    board_list = blf_df['Flop'][loc] + \
        [blf_df['Turn'][loc], blf_df['River'][loc]]
    hand_sb_list = ast.literal_eval(blf_df['SB cards'][loc].\
        replace("[ ", "['").replace(" ]", "']").replace(", ", "', '"))
    hand_bb_list = ast.literal_eval(blf_df['BB cards'][loc].\
        replace("[ ", "['").replace(" ]", "']").replace(", ", "', '"))

    handSB = [Card.new(card) for card in hand_sb_list]
    handBB = [Card.new(card) for card in hand_bb_list]
    board = [Card.new(card) for card in board_list]


    sb_per = 1 - eval.get_rank_percentage(eval.evaluate(handSB, board))
    bb_per = 1 - eval.get_rank_percentage(eval.evaluate(handBB, board))

    blf_df['SB Bluff Metric'][loc] = sb_per*blf_df['aggSB'][loc]
    blf_df['BB Bluff Metric'][loc] = bb_per*blf_df['aggBB'][loc]
blf_df


,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,...,River,SB stack,BB stack,SB cards,BB cards,Folded pre,aggSB,aggBB,SB Bluff Metric,BB Bluff Metric
0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]","[5c, 9c, 5s]",...,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]",False,1.561169,0.845973,1.132695,0.372650
1,9,603607691,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,MARUSYA94,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[9d, 4d, 9s]",...,6h,1.20,0.80,"[ 5c, 4c ]","[ 3h, 7s ]",False,0.780585,0.845973,0.320414,0.511416
2,14,603607964,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,gunchozaurs,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[Qs, 8s, 5s]",...,4c,2.63,2.01,"[ Kh, 9c ]","[ 2c, Jh ]",False,0.780585,0.845973,0.584026,0.638051
3,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[6s, 7s, 5h]",...,2d,2.65,2,"[ 5c, Ad ]","[ 6h, 9h ]",False,1.561169,0.845973,0.663633,0.358932
4,33,1180858754,Berkeley-0.01-0.02-USD-NoLimitHoldem-Pacific-6...,trebs15,turron3618,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[5d, Qd, 6d]",...,Jd,2.47,0.72,"[ 4c, Qh ]","[ 9h, 8c ]",False,0.780585,0.845973,0.184842,0.439198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77646,520797,553224321,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]","[6c, Jc, 3d]",...,9c,0.96,0.89,"[ 5d, Jh ]","[ Ad, As ]",False,1.561169,0.845973,0.870548,0.389542
77647,520798,553224342,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (r, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[7s, Jd, 5h]",...,Js,1,1,"[ 9s, Qd ]","[ 6c, Ac ]",False,0.780585,0.845973,0.428265,0.455751
77648,520799,553224357,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,roghnov,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[Jd, As, Qd]",...,Ts,1.04,0.96,"[ 9s, Ad ]","[ Qh, 7s ]",False,0.780585,0.845973,0.260578,0.308482
77649,520800,553224367,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[4d, 2d, Js]",...,Ts,1.10,0.90,"[ Qc, Kc ]","[ 9s, Ks ]",False,0.780585,0.845973,0.422721,0.459152


In [86]:
# Check if script version of code worked
script_df = pd.read_pickle('/home/jtatka/WinningEdge/poker_df_w_bluff.pkl')
script_df

,index,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,...,River,SB stack,BB stack,SB cards,BB cards,Folded pre,aggSB,aggBB,SB Bluff Metric,BB Bluff Metric
0,1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]","[5c, 9c, 5s]",...,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]",False,1.561169,0.845973,1.132695,0.372650
1,9,603607691,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,MARUSYA94,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[9d, 4d, 9s]",...,6h,1.20,0.80,"[ 5c, 4c ]","[ 3h, 7s ]",False,0.780585,0.845973,0.320414,0.511416
2,14,603607964,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,grandiozzo,gunchozaurs,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[Qs, 8s, 5s]",...,4c,2.63,2.01,"[ Kh, 9c ]","[ 2c, Jh ]",False,0.780585,0.845973,0.584026,0.638051
3,15,603607982,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,gunchozaurs,grandiozzo,"[(r, 0.03), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.02), (c, 0.02)]","[6s, 7s, 5h]",...,2d,2.65,2,"[ 5c, Ad ]","[ 6h, 9h ]",False,1.561169,0.845973,0.663633,0.358932
4,33,1180858754,Berkeley-0.01-0.02-USD-NoLimitHoldem-Pacific-6...,trebs15,turron3618,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[5d, Qd, 6d]",...,Jd,2.47,0.72,"[ 4c, Qh ]","[ 9h, 8c ]",False,0.780585,0.845973,0.184842,0.439198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77646,520797,553224321,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,silviu2010,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(b, 0.02), (c, 0.02)]","[(b, 0.02), (c, 0.02)]","[(b, 0.06), (c, 0.06)]","[6c, Jc, 3d]",...,9c,0.96,0.89,"[ 5d, Jh ]","[ Ad, As ]",False,1.561169,0.845973,0.870548,0.389542
77647,520798,553224342,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (r, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[7s, Jd, 5h]",...,Js,1,1,"[ 9s, Qd ]","[ 6c, Ac ]",False,0.780585,0.845973,0.428265,0.455751
77648,520799,553224357,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,roghnov,Sensei.Dbb,"[(c, 0.01), (k, 0)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (b, 0.02), (c, 0.02)]","[(k, 0), (k, 0)]","[Jd, As, Qd]",...,Ts,1.04,0.96,"[ 9s, Ad ]","[ Qh, 7s ]",False,0.780585,0.845973,0.260578,0.308482
77649,520800,553224367,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[4d, 2d, Js]",...,Ts,1.10,0.90,"[ Qc, Kc ]","[ 9s, Ks ]",False,0.780585,0.845973,0.422721,0.459152
